## From https://discourse.julialang.org/t/solving-sparse-linear-systems-fast/83071/9

In [1]:
using LinearSolve, LinearSolvePardiso, SparseArrays
using AlgebraicMultigrid
using IncompleteLU

In [2]:
Threads.nthreads()

4

In [3]:
#..construct the mesh: see before 
N = 100; h = 1/N; 
x = Vector(0:h:1); 

#..Mesh with points and edges 
#..point holds the coordinates of the left and right node of the element
#..edges holds the global indices of the left and right node of the element
points = collect( [x[i], x[i+1]] for i in 1:length(x)-1) 
edges = collect( [i, i+1] for i in 1:length(x)-1); 

#..Set the source function 
fsource(x) = x*(x-1); 

#..Initialize global matrix and right-hand side value 
A = zeros(length(x), length(x)); 
f = zeros(length(x), 1); 

#..Perform loop over elements and assemble global matrix and vector 
for i=1:length(edges) 

  xl, xr = points[i,:][1]
  floc = (xr-xl) * [fsource(xl) fsource(xr)];
  Aloc = (1/(xr-xl))*[1 -1; -1 1]; 

  for j=1:2 
    f[edges[i][j]] += floc[j];
    for k =1:2 
      A[edges[i][j], edges[i][k]] += Aloc[j,k]; 
    end 
  end 

end

In [4]:
#..handle the boundary conditions in the matrix and right-hand side vector 
A[1,1] = 1;     A[1,2] = 0;        f[1]   = 0; 
A[end,end-1]=0; A[end,end] = 1;    f[end] = 0;

f_orig = copy(f)
A_orig = copy(A);

In [5]:
#..solve the linear system
u = A \ f

101×1 Matrix{Float64}:
  2.3236967905404526e-15
 -0.0016664999999976122
 -0.003331019999997548
 -0.004991619999997483
 -0.006646399999997419
 -0.008293499999997353
 -0.009931099999997289
 -0.011557419999997225
 -0.013170719999997159
 -0.014769299999997095
 -0.01635149999999703
 -0.017915699999996967
 -0.0194603199999969
  ⋮
 -0.01791569999999904
 -0.016351499999999124
 -0.01476929999999921
 -0.013170719999999296
 -0.011557419999999385
 -0.00993109999999949
 -0.008293499999999576
 -0.00664639999999966
 -0.004991619999999745
 -0.0033310199999998305
 -0.001666499999999915
  0.0

In [6]:
A = sparse(A_orig)  # your matrix
b = copy(f_orig[:,1])  # test b

prob = LinearProblem(A, b)
prob1 = prob;

In [7]:
u = A \ b

101-element Vector{Float64}:
  0.0
 -0.0016664999999999946
 -0.0033310199999999892
 -0.004991619999999982
 -0.006646399999999976
 -0.00829349999999997
 -0.009931099999999964
 -0.01155741999999996
 -0.013170719999999955
 -0.014769299999999952
 -0.01635149999999995
 -0.017915699999999948
 -0.019460319999999948
  ⋮
 -0.017915699999999996
 -0.016351499999999995
 -0.014769299999999996
 -0.013170719999999997
 -0.011557419999999997
 -0.009931100000000014
 -0.008293500000000011
 -0.006646400000000009
 -0.004991620000000006
 -0.003331020000000004
 -0.001666500000000002
  0.0

In [8]:
for alg in (
    MKLPardisoFactorize(),
    MKLPardisoIterate(),
    UMFPACKFactorization(),
    KLUFactorization())

    u = solve(prob, alg).u
    println(u)
end

[0.0, -0.0016664999999998766, -0.0033310199999997533, -0.004991619999999629, -0.006646399999999505, -0.008293499999999381, -0.009931099999999258, -0.011557419999999135, -0.013170719999999012, -0.014769299999998889, -0.016351499999998766, -0.017915699999998643, -0.019460319999998518, -0.0209838199999984, -0.022484699999998286, -0.023961499999998165, -0.02541279999999806, -0.026837219999997955, -0.02823341999999784, -0.029600099999997735, -0.030935999999997632, -0.03223989999999753, -0.033510619999997444, -0.03474701999999735, -0.03594799999999726, -0.03711249999999716, -0.038239499999997074, -0.039328019999997, -0.040377119999996915, -0.04138589999999684, -0.042353499999996776, -0.04327909999999671, -0.044161919999996656, -0.045001219999996595, -0.04579629999999653, -0.04654649999999647, -0.04725119999999644, -0.0479098199999964, -0.048521819999996364, -0.04908669999999634, -0.049603999999996304, -0.05007329999999627, -0.05049421999999626, -0.05086641999999624, -0.05118959999999624, -0.

In [9]:
ml = ruge_stuben(A) # Construct a Ruge-Stuben solver
pl = aspreconditioner(ml)
solve(prob1, KrylovJL_GMRES(), Pl = pl).u

101-element Vector{Float64}:
  0.00017927577243907191
  2.918115309387788e-6
 -8.722057429824132e-5
 -0.0002417563708244095
 -0.0004669576293408065
 -0.0007712603051948755
 -0.001172833241635216
 -0.0016727025927204418
 -0.002292397079474467
 -0.0030207044762882275
 -0.0038689466022032055
 -0.004809046174739953
 -0.005844824211762348
  ⋮
 -0.013903005457485679
 -0.012477721384131231
 -0.01103023717966654
 -0.009580247123391644
 -0.008107501576575528
 -0.006647438801141328
 -0.0051932118595887115
 -0.0037801701078117728
 -0.0024058215641414114
 -0.0011245542565435209
  1.9893134639941097e-5
  0.001299490664217812

In [10]:
pl = ilu(A, τ = 0.1) # τ needs to be tuned per problem
solve(prob1, KrylovJL_GMRES(), Pl = pl).u

101-element Vector{Float64}:
  0.0
 -0.001666499999999895
 -0.00333101999999979
 -0.004991619999999685
 -0.0066463999999995796
 -0.008293499999999477
 -0.009931099999999374
 -0.011557419999999274
 -0.013170719999999173
 -0.014769299999999068
 -0.01635149999999897
 -0.017915699999998872
 -0.01946031999999877
  ⋮
 -0.01791569999999933
 -0.016351499999999387
 -0.014769299999999447
 -0.013170719999999504
 -0.011557419999999563
 -0.00993109999999964
 -0.008293499999999697
 -0.006646399999999757
 -0.004991619999999818
 -0.0033310199999998795
 -0.0016664999999999397
  0.0